In [1]:
import pandas as pd
import glob
import re
import math
import urllib.request
from bs4 import BeautifulSoup
from operator import itemgetter
import csv

In [45]:
def native_vote_csv_df():
    filepath = "/Users/tdt62/Desktop/test_data/2018_10_*vote*"
    list_ = []

    # Takes all of the csv file and makes one big dataframe
    for name in glob.glob(filepath):
        df = pd.read_csv(name,index_col=None, header=0)
        list_.append(df)

#     filepath = "/Users/tdt62/Desktop/test_data/2018_11_0[0-5]*vote*"

#     # Takes all of the csv file and makes one big dataframe
#     for name in glob.glob(filepath):
#         df = pd.read_csv(name,index_col=None, header=0)
#         list_.append(df)

    # Makes the big df in memory
    frame = pd.concat(list_, axis = 0, ignore_index = True)
    frame.fillna("NA", inplace=True)
    return frame

def news_deserts_csv_df():
    filepath = "/Users/tdt62/Desktop/test_data/2018_10_*clean*"
    list_ = []

    # Takes all of the csv file and makes one big dataframe
    for name in glob.glob(filepath):
        df = pd.read_csv(name,index_col=None, header=0, delimiter='\t')
        list_.append(df)

    filepath = "/Users/tdt62/Desktop/test_data/2018_11_0[0-5]*clean*"

    # Takes all of the csv file and makes one big dataframe
    for name in glob.glob(filepath):
        df = pd.read_csv(name,index_col=None, header=0, delimiter='\t')
        list_.append(df)

    # Makes the big df in memory
    frame = pd.concat(list_, axis = 0, ignore_index = True)
    frame.fillna("NA", inplace=True)
    return frame

def total_tweets(df):

    # Gets the total number of tweets
    total_tweets_num = df.shape[0]  # gives number of row count
    return total_tweets_num

def unique_tweets(df):

    def run_isdigit(x):
        if math.isnan(x):
            return False
        else:
            return x.isdigit()

    # Gets the unique tweets i.e. no retweeted status
    df['rt_isdigit'] = list(map(lambda x: x.isdigit(), df['Retweeted_Status']))
    unique_df = df.loc[df['rt_isdigit'] == False]
    unique_tweet_num = unique_df.shape[0]
    return unique_tweet_num

def unique_domains(df):
    # Checks if URLs is empty or not
    def empty_list_check(x):
        if(x != '[]'):
            return True
        else:
            return False

    def get_domains(full_domain):
        if(isinstance(full_domain, str)):
            m = full_domain.split("//")[-1].split("/")[0].split('?')[0]
            return m

        else:
            print(full_domain)
            return full_domain


    # Gets the tweets that have URLs in them i.e. URLs column is not '[]'
    df['domains'] = list(map(lambda x: empty_list_check(x), df['URLs']))

    # Gets the domains to be only the domains i.e. www.xyz.com
    df['actual_domain'] = list(map(lambda x: get_domains(x), df['URLs']))

    # Creates a df that only has domains in the tweet
    unique_df = df.loc[df['domains'] == True]

    # Get the number of domains
    unique_domains = len(unique_df.actual_domain.unique())

    domains_df = unique_df

    return unique_domains, domains_df

def unique_users(df):
    unique_users = len(df.User_ID.unique())
    return unique_users

def original_content_user(df):

    def run_isdigit(x):
        if math.isnan(x):
            return False
        else:
            return x.isdigit()

    # Gets the unique tweets i.e. no retweeted status
    df['rt_isdigit'] = list(map(lambda x: x.isdigit(), df['Retweeted_Status']))
    unique_df = df.loc[df['rt_isdigit'] == False]
    unique_users = len(unique_df.User_ID.unique())
    return unique_users

def top_news_domains():

    #specify the url
    news_source = "https://www.alexa.com/topsites/category/Top/News"

    #Query the website and return the html to the variable 'page'
    page = urllib.request.urlopen(news_source)

    #Parse the html in the 'page' variable, and find all of the website domains
    soup = BeautifulSoup(page, features="html.parser")
    links = soup.find_all("a")
    link_list = []
    actual_news_list = []
    links_dict = {}
    for link in links:
        link_list.append(link.get("href"))
    i = 0
    for link in link_list:
        if link:
            if "siteinfo/" in link:
                news_link = link.split("/siteinfo/")[1]
                links_dict[news_link] = 0
                actual_news_list.append(news_link)

    return actual_news_list, links_dict

def get_top_news_domains(df, news_list, news_dict, top_val):

    # Function to iterate over the dataframe
    def iterate_domains_dict(x):
        for element in x.split():
            if element in news_list:
                news_dict[x] += 1
        return 0


    top_list = []

    # Iterates over the domain column
    list(map(lambda x: iterate_domains_dict(x), df['actual_domain']))

    # Gets the sorted news_list in descending order
    sorted_news_list = (list(sorted(news_dict.items(), key=itemgetter(1), reverse=True)))

    # Gets the top 20 results of the sorted list
    sorted_news_list = sorted_news_list[0:top_val]

    # Stores all the names in a list, so we can read correctky
    for item in sorted_news_list:
        top_list.append(item[0])

    return top_list, sorted_news_list

def get_top_domains(df, top_val):
    top_list = []
    domain_dict = {}
    
    def iterate_domains(x):
        domain_list = list(x.split())
        for element in domain_list:
            if element in domain_dict:
                domain_dict[element] += 1
            else:
                domain_dict[element] = 1
    
    # Create the hashtag dict
    list(map(lambda x: iterate_domains(x), df['actual_domain']))
    
    # Gets the sorted news_list in descending order
    sorted_domain_list = (list(sorted(domain_dict.items(), key=itemgetter(1), reverse=True)))
    
    # Gets the top 20 results of the sorted list
    sorted_domain_list = sorted_domain_list[0:top_val]

    # Stores all the names in a list, so we can read correctky
    for item in sorted_domain_list:
        top_list.append(item[0])

    return top_list, sorted_domain_list
    
def get_number_of_domains(df):
    total_number_domains = len(df.actual_domain)
    return total_number_domains

def count_total_news(news_dict):
    total = sum(list(news_dict.values()))
    return total

def make_domain_csv(top_news_dict):
    with open('news_domains.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in top_news_dict.items():
            writer.writerow([key, value])

def make_hashtag_csv(hashtag_dict):
    with open('top_hashtags.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in top_news_dict.items():
            writer.writerow([key, value])

def make_domain_csv(domain_dict):
    with open('top_domains.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in top_news_dict.items():
            writer.writerow([key, value])

def hashtags(df, top_val):
    
    top_list = []
    hashtag_dict = {}
    
    def iterate_hashtags(x):
        hashtag_list = list(x.split(', '))
        for element in hashtag_list:
            if element.strip('[]') in hashtag_dict and x != '[]':
                hashtag_dict[element.strip('[]')] += 1
            else:
                hashtag_dict[element.strip('[]')] = 1
    
    # Create the hashtag dict
    list(map(lambda x: iterate_hashtags(x), df['Hashtags']))
    
    # Gets the sorted news_list in descending order
    sorted_news_list = (list(sorted(hashtag_dict.items(), key=itemgetter(1), reverse=True)))
    
    # Gets the top 20 results of the sorted list
    sorted_news_list = sorted_news_list[0:top_val]

    # Stores all the names in a list, so we can read correctky
    for item in sorted_news_list:
        top_list.append(item[0])

    return top_list, sorted_news_list

def top_user_mentions(df, top_val):
    
    top_users_list = []
    hashtag_dict = {}
    
    def iterate_user_mentions(x):
        
        hashtag_list = list(x.split())
        for element in hashtag_list:
            if element.strip('{}:,').isdigit():
                continue
            elif element.strip('{}:') in hashtag_dict and element != '{}' and not element.strip('{}:').isdigit():
                hashtag_dict[element.strip('{}:')] += 1
            else:
                hashtag_dict[element.strip('{}:')] = 1
    
    # Create the hashtag dict
    list(map(lambda x: iterate_user_mentions(x), df['User_Mentions']))
    
    # Gets the sorted news_list in descending order
    sorted_news_list = (list(sorted(hashtag_dict.items(), key=itemgetter(1), reverse=True)))
    
    # Gets the top 20 results of the sorted list
    sorted_news_list = sorted_news_list[0:top_val]

    # Stores all the names in a list, so we can read correctky
    for item in sorted_news_list:
        top_users_list.append(item[0])

    return top_users_list, sorted_news_list

In [3]:
native_df = native_vote_csv_df()
native_df

,TweetID,Timestamp,Full_Text,In_Reply_To_User_ID,User_ID,User_Name,User_Screen_Name,Coordinates,Place,Bounding_Box,Quoted_Status_ID,Retweeted_Status,Hashtags,URLs,User_Mentions,Media,Language
0,1052544860421525504,2018-10-17 13:00:24,b'Want to understand the Green Party? Sara Mae...,NA,14345531,Liane Hernandez,loquesera,NA,NA,[],0,[],[],"[{'url': 'https://t.co/LSXEF5e6oR', 'expanded_...",[],[],en
1,1052544901513007105,2018-10-17 13:00:34,b'Follow Indigenous people. Listen. @rebeccana...,NA,920171445484896256,Evil ginger bitch,iamapartsguy,NA,NA,[],0,1052539455712751616,[],[],"[{'screen_name': 'KaitlinCurtice', 'name': 'Ka...",[],en
2,1052544911742889984,2018-10-17 13:00:36,b'See our AARP poll of adults over 50+ in WA C...,NA,617089370,Doug Shadel,dougshadel,NA,NA,[],0,1052254768096104448,[],[],"[{'screen_name': 'AARPWA', 'name': 'AARP Washi...",[],en
3,1052544988914049024,2018-10-17 13:00:55,b'@senatemajldr @GOP @TomColeOK04 @JimInhofe y...,1.24998e+09,978787610,Adrienne #VoteBlue Young #Resist,gentlemindmom,NA,NA,[],1052249777864409088,[],[],"[{'url': 'https://t.co/jBEWvUx9tD', 'expanded_...","[{'screen_name': 'senatemajldr', 'name': 'Lead...",[],en
4,1052545025534496768,2018-10-17 13:01:04,b'@KaitlinCurtice @rebeccanagle @xodanix3 @jus...,2.23787e+09,15718981,Jordan Humm,__jordanhumm,NA,NA,[],0,[],[],"[{'url': 'https://t.co/A5pv6Kiflo', 'expanded_...","[{'screen_name': 'KaitlinCurtice', 'name': 'Ka...",[],en
5,1052545034506125312,2018-10-17 13:01:06,"b'@ShariceDavids is a 2016 White House fellow,...",NA,917515800,Marcie,_strate,NA,NA,[],0,1052463048743690240,[],[],"[{'screen_name': 'RavenResists', 'name': '👻🎃🦇🕸...",[],en
6,1052545036058013696,2018-10-17 13:01:06,b'This story is why I am voting for @shariceda...,NA,874065317173096453,Maddie Barry,MaddieBarry8,NA,NA,[],0,1052519024217784320,[],[],"[{'screen_name': 'gayleasher1', 'name': 'Gayle...",[],en
7,1052545069532626944,2018-10-17 13:01:14,b'',NA,816793158293987328,Jderb,jjderb41,NA,NA,[],0,1052024528103063552,"[{'text': 'walkaway', 'indices': [45, 54]}, {'...",[],"[{'screen_name': 'ledermanu', 'name': 'Uri Led...",[],en
8,1052545083315228672,2018-10-17 13:01:17,b'@POTUS @realDonaldTrump @VP @RepTomEmmer @Re...,NA,4298028747,Patricia Hammes,PatriciaHammes2,NA,NA,[],1052542592599162880,1052543537273851904,[],[],"[{'screen_name': 'MinnesotaMiners', 'name': 'M...",[],und
9,1052545104542617600,2018-10-17 13:01:22,b'@ShariceDavids She is Cornell Law School gra...,NA,917515800,Marcie,_strate,NA,NA,[],0,1052460609319698432,[],[],"[{'screen_name': 'RavenResists', 'name': '👻🎃🦇🕸...",[],en


In [43]:
news_df = news_deserts_csv_df()
news_df

,Unnamed: 0,TweetID,Timestamp,Full_Text,In_Reply_To_User_ID,User_ID,User_Name,User_Screen_Name,Coordinates,Place,Bounding_Box,Quoted_Status_ID,Retweeted_Status,Hashtags,URLs,User_Mentions,Media,Language
0,0,1052544755572211712,2018-10-17 12:59:59,b'',NA,4708477574,Only_Suju,apr_sj,NA,NA,[],0,1052491599190482944,"['supershow7', 'TheConcertTour', 'pcas']",[],{'sarangsapphire': 149126998},[],en
1,1,1052544756146995200,2018-10-17 12:59:59,b'',NA,742434618058641412,Jan,MissJanisinNC,NA,NA,[],0,1052173050211393536,[],[],{'Freeyourmindkid': 120995738},[],en
2,2,1052544756092407808,2018-10-17 12:59:59,b'Blue Wave? Hell no! democrats are disintegra...,NA,1022741765286187008,Végh Lajosné,LajosneVegh,NA,NA,[],1052410293421268992,1052440066281353216,['ILGOV'],[],{'GmanFan45': 4158758146},[],en
3,3,1052544756239224832,2018-10-17 12:59:59,b'',NA,37737068,john napolitano,snappynappy,NA,NA,[],1052339071756976133,1052343825862156288,[],['https://twitter.com/PrissysTruth/status/1052...,{'Fuctupmind': 139499789},[],en
4,4,1052544756428021760,2018-10-17 12:59:59,b'Adam ded\xc4\xb1\xc4\x9fin hayaller kurar ha...,NA,752233055054684160,İremhelvacıoğlusevdalısı💕💕,nuraytosun316,NA,NA,[],0,[],"['TahirKaleli', 'SenAnlatKaradeniz', 'LifeLine']",[],{},[],tr
5,5,1052544756016865280,2018-10-17 12:59:59,b'',NA,716051051,Votes at 12 Campaign,votesat12,NA,NA,[],1052541968096595968,1052544127479504896,[],['https://twitter.com/mattsingh_/status/105254...,"{'votesat12': 716051051, 'christiancalgie': 21...",[],en
6,6,1052544756499304449,2018-10-17 12:59:59,b'The Left Hates @FoxNews!!! The Only Power Ba...,NA,841692239562653697,Teri Saunders,Crowntiptoe,NA,NA,[],0,1052525835448668160,[],[],"{'FoxNews': 1367531, 'DFBHarvard': 80634786507...",[],en
7,7,1052544756386029568,2018-10-17 12:59:59,b'',NA,970402736582873089,Sarah Campos,gamora4ever,NA,NA,[],0,1050480117548507137,"['TaylorSwift', 'TheFemaleArtist', 'PCAs']",[],"{'taylornation13': 80144576, 'tanya1992x': 719...","[{'id': 1050480110976032768, 'id_str': '105048...",en
8,8,1052544756457201664,2018-10-17 12:59:59,b'This is a fight for the soul of our nation. ...,NA,2229608989,meagan mangum,MangumMeagan,NA,NA,[],0,1052526714419666944,[],[],{'JeffMerkley': 223166587},[],en
9,9,1052544756360736768,2018-10-17 12:59:59,b'[TUTORIAL THREAD] 7th Yegreen Musical Awards...,NA,1542729920,rebecca,hana_rebeca,NA,NA,[],0,1052429480600440832,[],[],{'suhounionglobal': 776968690445103104},[],en


In [41]:
top_users_list, sorted_news_list = top_user_mentions(news_df, 20)
print(top_list)

["'realDonaldTrump'", "'POTUS'", "'GOP'", "'DonaldJTrumpJr'", "'BarackObama'", "'BTS_twt'", "'RealJamesWoods'", "'AndrewGillum'", "'BetoORourke'", "'tedcruz'", "'charliekirk11'", "'btsvotingteam'", "'attn'", "'mitchellvii'", "'RonDeSantisFL'", "'DebbieStabenow'", "'rihanna'", "'FoxNews'", "'MichaelAvenatti'", "'fred_guttenberg'"]


In [49]:
# Create big df for manipulating the data
news_df = news_deserts_csv_df()

# Get the count of unique domains and the df associated
unique_domains_count, domains_df = unique_domains(news_df)

# Total number of tweets 
count_row = total_tweets(news_df)

# Unique number of tweets
unique_tweets_count = unique_tweets(news_df)

# Unique users based on UserID
unique_users_count = unique_users(news_df)

# Original Content creater count
original_content = original_content_user(news_df)

# Generates the list and dict associated with the domains
top_news_list, top_news_dict = top_news_domains()

# Gets the top 10 news domains
top_news_list, sorted_top_news_list = get_top_news_domains(domains_df, top_news_list, top_news_dict, 10)

# Gets top 20 domains
top_domain_list, sorted_top_news_list = get_top_domains(domains_df, 20)

# Gets the total number of domains mentioned
total_domains = get_number_of_domains(domains_df)

# Count total number of news domains
total_news_domains = count_total_news(top_news_dict)

# Generate a CSV for the domain and it's occurence
make_domain_csv(top_news_dict)

# Gets the list for top 20 hashtags
top_hashtags, hashtag_sorted_list = hashtags(news_df, 20)

# Get the top 20 user mentions
top_users_list, sorted_news_list = top_user_mentions(news_df, 20)

# Prints each value
print("Total Number of Tweets: {}".format(count_row))
print("Unique Tweets: {}".format(unique_tweets_count))
print("Unique Domains: {}".format(unique_domains_count))
print("Unique Users: {}".format(unique_users_count))
print("Original Content Users: {}".format(original_content))
print("Total Number of Domains: {}".format(total_domains))
print("Total Number of News Domains: {}\n".format(total_news_domains))
print("Top 10 News Domains: {}\n".format(top_news_list))
print("Top 20 Hashtags: {}\n".format(top_hashtags))
print("Top 20 Domains: {}\n".format(top_domain_list))
print("Top 20 User Mentions: {}\n".format(top_users_list))

Total Number of Tweets: 517257
Unique Tweets: 124850
Unique Domains: 6234
Unique Users: 283794
Original Content Users: 97142
Total Number of Domains: 132976
Total Number of News Domains: 980

Top 10 News Domains: ['thehill.com', 'abcnews.go.com', 'nypost.com', 'time.com', 'variety.com', 'weather.com', 'indianexpress.com', 'nbcnews.com', 'reddit.com', 'bloomberg.com']

Top 20 Hashtags: ["'PCAs'", "'BTS'", "'MAGA'", "'Shadowhunters'", "'IDOL'", "'TaylorSwift'", "'vote'", "'Vote'", "'TexasDebate'", "'TheGroup'", "'VoteRed'", "'TheFemaleArtist'", "'VoteRedToSaveAmerica'", "'POTUS'", "'PresidentTrump'", "'pcas'", "'TheSong'", "'VoteBlue'", "'WalkAway'", "'Democrats'"]

Top 20 Domains: ['twitter.com', 'bit.ly', 'www.breitbart.com', 'politics.myajc.com', 'www.instagram.com', 'youtu.be', 'www.nytimes.com', 'www.cnn.com', 'www.washingtonpost.com', 'gop.cm', "2018mama.com']", 'thehill.com', 'ift.tt', 'www.foxnews.com', 'sai-zen-sen.jp', 'dlvr.it', 'dailycaller.com', 'fxn.ws', 'www.dailywire.com'

In [7]:
unique_domains, domains_df = unique_domains(native_df)
count_row = total_tweets(native_df)
unique_tweets_count = unique_tweets(native_df)
unique_users_count = unique_users(native_df)
original_content = original_content_user(native_df)
top_news_list, top_news_dict = top_news_domains()
top_news_list, sorted_top_news_list = get_top_domains(domains_df, top_news_list, top_news_dict, 10)

print("Total Number of Tweets: {}".format(count_row))
print("Unique Tweets: {}".format(unique_tweets_count))
print("Unique Domains: {}".format(unique_domains))
print("Unique Users: {}".format(unique_users_count))
print("Original Content Users: {}".format(original_content))
print("Top 10 News Domains: {}".format(top_news_list))

TypeError: get_top_domains() takes 2 positional arguments but 4 were given